In [1]:
import numpy as np
from mido import MidiFile

In [2]:
!ls -lah

total 72
drwxr-xr-x  9 rsilvei  813930708   288B May  7 22:04 .
drwxr-xr-x  8 rsilvei  813930708   256B May  7 15:45 ..
drwxr-xr-x  3 rsilvei  813930708    96B May  7 10:29 .ipynb_checkpoints
-rw-r--r--  1 rsilvei  813930708    16K May  7 22:04 MIDI_converter.ipynb
-rw-r--r--@ 1 rsilvei  813930708   570B May  7 14:37 durations.txt
-rw-r--r--  1 rsilvei  813930708   118B May  7 21:41 durations_oldMacDo.txt
-rw-r--r--@ 1 rsilvei  813930708   236B May  7 14:37 freqs.txt
-rw-r--r--  1 rsilvei  813930708   236B May  7 21:41 freqs_oldMacDo.txt
-rw-r--r--@ 1 rsilvei  813930708   693B May  7 10:29 old_macdonaldPNO.mid


In [3]:
mid = MidiFile('old_macdonaldPNO.mid', clip=True)
print(mid)

<midi file 'old_macdonaldPNO.mid' type 1, 2 tracks, 137 messages>


In [4]:
print(mid.ticks_per_beat)

960


In [5]:
def convertVelocityDuration(velocity):
    base_duration_ms = 10 # 10 ms duration
    return 5 + int(10 * velocity/127)

In [6]:
def noteToFreq(note):
    a = 440 #frequency of A (coomon value is 440Hz)
    return int((a / 32) * (2 ** ((note - 9) / 12)))

In [7]:
def convert_hex_int(h):
    hex_string = "0x" + h
    return int(hex_string, 16)

In [8]:
FREQ_DIVIDER = 1

In [12]:
mid.ticks_per_beat

960

In [16]:
def convertMidiArduino(track, freq_file, duration_file):
    freqs_list = []
    durations_list = []
    freqs_file = open(freq_file, 'w')
    durations_file  = open(duration_file, 'w')
    for i, msg in enumerate(track):
        print(msg)
        commands = [convert_hex_int(x) for x in msg.hex().split()]
        if commands[0] == 144:
            note = commands[1]
            freq = noteToFreq(note)
            freqs_list.append(freq)
            freqs_file.write(str(freq) + ",")
        elif commands[0] == 128:
            duration = int(msg.time/mid.ticks_per_beat)
            durations_file.write(str(duration) + ",")
            durations_list.append(duration)
    freqs_file.close()
    durations_file.close()
    return freqs_list, durations_list

In [17]:
f, d = convertMidiArduino(mid.tracks[1], "freqs_oldMacDo.txt", "durations_oldMacDo.txt")

program_change channel=0 program=0 time=0
control_change channel=0 control=121 value=0 time=0
control_change channel=0 control=64 value=0 time=0
control_change channel=0 control=91 value=48 time=0
control_change channel=0 control=10 value=51 time=0
control_change channel=0 control=7 value=100 time=0
note_on channel=0 note=67 velocity=66 time=0
control_change channel=0 control=121 value=0 time=0
control_change channel=0 control=64 value=0 time=0
control_change channel=0 control=91 value=48 time=0
control_change channel=0 control=10 value=51 time=0
control_change channel=0 control=7 value=100 time=0
<meta message track_name name='Piano' time=0>
note_off channel=0 note=67 velocity=0 time=960
note_on channel=0 note=67 velocity=75 time=0
note_off channel=0 note=67 velocity=0 time=960
note_on channel=0 note=67 velocity=78 time=0
note_off channel=0 note=67 velocity=0 time=960
note_on channel=0 note=62 velocity=66 time=0
note_off channel=0 note=62 velocity=0 time=960
note_on channel=0 note=64 

In [19]:
!cat freqs_oldMacDo.txt

391,391,391,293,329,329,293,493,493,440,440,391,293,391,391,391,293,329,329,293,493,493,440,440,391,293,293,391,391,391,293,293,391,391,391,391,391,391,391,391,391,391,391,391,391,391,391,391,391,391,293,329,329,293,493,493,440,440,391,

In [20]:
!cat durations_oldMacDo.txt

1,1,1,1,1,1,0,1,1,1,1,2,0,1,1,1,1,1,1,0,1,1,1,1,2,0,0,1,1,1,0,0,1,1,2,0,0,1,0,0,1,0,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,2,

## Source
- https://www.audiocontentanalysis.org/code/helper-functions/midi-pitch-to-frequency-conversion-2/
- https://www.cs.cmu.edu/~music/cmsip/readings/MIDI%20tutorial%20for%20programmers.html  
- https://www.elektormagazine.com/index.php/files/attachment/331
- [Great tutorial](https://arduinoplusplus.wordpress.com/2018/05/11/playing-midi-files-on-arduino-part-1-standard-midi-files/)  
- http://subsynth.sourceforge.net/midinote2freq.html
- https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c